In [ ]:
# /share/hariharan/ck696/allclear_0529_lllll/baselines/DiffCR3/models/palette_convnext_ca

In [2]:
print("Begin srcipt")
import sys, os
if "ck696" in os.getcwd():
    sys.path.append("/share/hariharan/ck696/allclear/baselines/DiffCR3")
else:
    sys.path.append("/share/hariharan/cloud_removal/allclear/baselines/DiffCR3")
os.environ["WANDB_API_KEY"] = "5f04d2ce100707f23b71379f67f28901d496edda"
# os.environ["WANDB_MODE"] = "disabled"

import argparse
import os
import warnings
import torch
import torch.multiprocessing as mp
import wandb


from core.logger import VisualWriter, InfoLogger
import core.praser as Praser
import core.util as Util
from data import define_dataloader
from models import create_model, define_network, define_loss, define_metric

print("Complete loading packages")

parser = argparse.ArgumentParser()

parser.add_argument('-c', '--config', type=str, default='config/ours_sigmoid.json', help='JSON file for configuration')
# parser.add_argument('-c', '--config', type=str, default='config/ours_sigmoid.json', help='JSON file for configuration')
parser.add_argument('-p', '--phase', type=str, choices=['train','test'], help='Run train or test', default='test')
parser.add_argument('-b', '--batch', type=int, default=2, help='Batch size in every gpu')
parser.add_argument('-gpu', '--gpu_ids', type=str, default="0")
parser.add_argument('-d', '--debug', action='store_true')
parser.add_argument('-P', '--port', default='21012', type=str)
parser.add_argument('--train_dataset', default='original', type=str, choices=["sen2mtc", "allclear"])

''' parser configs '''
args, _ = parser.parse_known_args()
opt = Praser.parse(args)

print(f"Using Dataset: {args.train_dataset}")

''' cuda devices '''
gpu_str = ','.join(str(x) for x in opt['gpu_ids'])
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_str
print('export CUDA_VISIBLE_DEVICES={}'.format(gpu_str))

''' use DistributedDataParallel(DDP) and multiprocessing for multi-gpu training'''
# [Todo]: multi GPU on multi machine
opt['world_size'] = 1 
# main_worker(0, 1, opt)

gpu = 0
ngpus_per_node = 1

Util.set_seed(opt['seed'])

''' set logger '''
phase_logger = InfoLogger(opt)
phase_writer = VisualWriter(opt, phase_logger)  
phase_logger.info('Create the log file in directory {}.\n'.format(opt['path']['experiments_root']))

run_name = f"DiffCR_{args.train_dataset}_bs{args.batch}_0526"
wandb.init(project="allclear-diffcr-v1", name=run_name, config=)


print('''set networks and dataset''')
'''set networks and dataset'''
phase_loader, val_loader = define_dataloader(phase_logger, opt) # val_loader is None if phase is test.

if args.train_dataset == "allclear":
    import sys, os
    if "ck696" in os.getcwd():
        sys.path.append("/share/hariharan/ck696/allclear")
    else:
        sys.path.append("/share/hariharan/cloud_removal/allclear")
    
    from dataset.dataloader_v1 import CRDataset
    from torch.utils.data import DataLoader, Dataset
    
    class CRDatasetWrapper(Dataset):
        def __init__(self, original_dataset):
            self.original_dataset = original_dataset
    
        def __len__(self):
            return 688
    
        def __getitem__(self, idx):
            batch = self.original_dataset[idx]
            # input_images, target = batch["input_images"], batch["target"], 
            cond_image = batch["input_images"][(1,2,3),...].reshape(9,256,256)
            gt_image = batch["target"][(1,2,3),...].reshape(3,256,256)
            return {"gt_image": gt_image, "cond_image": cond_image, "path": ["", ""]}
    
    import json
    with open('/share/hariharan/cloud_removal/metadata/v3/s2p_tx3_train_2k_v1.json') as f:
        metadata = json.load(f)
    #len=3168
    train_data = CRDataset(metadata, 
                        selected_rois="all", 
                        main_sensor="s2_toa", 
                        aux_sensors=[],
                        aux_data=["cld_shdw"],
                        format="stp",
                        target="s2p",
                        tx=3)
    wrapped_train_data = CRDatasetWrapper(train_data)
    phase_loader = DataLoader(wrapped_train_data, batch_size=args.batch, shuffle=True)

networks = [define_network(phase_logger, opt, item_opt) for item_opt in opt['model']['which_networks']]

print('''set metrics, loss, optimizer and  schedulers''')
''' set metrics, loss, optimizer and  schedulers '''
metrics = [define_metric(phase_logger, item_opt) for item_opt in opt['model']['which_metrics']]
losses = [define_loss(phase_logger, item_opt) for item_opt in opt['model']['which_losses']]

print('''set model''')
model = create_model(
    opt = opt,
    networks = networks,
    phase_loader = phase_loader,
    val_loader = None,
    losses = losses,
    metrics = metrics,
    logger = phase_logger,
    writer = phase_writer
)
model.save_current_results_flag = 0

# model.load_networks()

# params = torch.load("./pretrained/diffcr_new.pth")
# model.netG.load_state_dict(params,strict=False)
print('''begin training''')
model.train()

model.test()

Begin srcipt
Complete loading packages
Using Dataset: original
export CUDA_VISIBLE_DEVICES=0


wandb: Currently logged in as: ck696 (cornell-kao). Use `wandb login --relogin` to force relogin


set networks and dataset
set metrics, loss, optimizer and  schedulers
set model
begin training


  0%|          | 0/344 [00:00<?, ?it/s]/share/hariharan/ck696/allclear_0529_lllll/baselines/DiffCR3/core/logger.py:163: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._data.total[key] += value * n
/share/hariharan/ck696/allclear_0529_lll

In [ ]:
model.test()

In [ ]:
print(model.test_metrics.result())

In [ ]:
from tqdm.auto import tqdm
from glob import glob
import numpy as np
import cv2
from evaluation.psnr_ssim import calculate_psnr, calculate_ssim

PATH = "/share/hariharan/ck696/allclear/baselines/DiffCR3/experiments/test_ours_double_encoder_splitcaCond_splitcaUnet_sigmoid_240513_153841/results/test"
src = os.path.join(PATH, "0/GT")
dst = os.path.join(PATH, "0/Out")
psnr_list = []
ssim_list = []
for i, j in tqdm(zip(glob(src + "*"), glob(dst + "*")), total=len(glob(src + "*"))):
    img1 = cv2.imread(i)
    img2 = cv2.imread(j)
    psnr = calculate_psnr(img1, img2, 0)
    ssim = calculate_ssim(img1, img2, 0)
    psnr_list.append(psnr)
    ssim_list.append(ssim)
print("PSNR: ", np.mean(psnr_list))
print("SSIM: ", np.mean(ssim_list))

In [ ]:
model.save_current_results_flag = 0

In [ ]:
for batch in phase_loader: break

In [ ]:
batch.keys(), batch["gt_image"].shape, batch["cond_image"].shape

In [ ]:
batch["path"]

In [ ]:
# for ac_batch in train_loader:
#     break
for diffcr_batch in wrapped_train_loader:
    break

In [ ]:
ac_batch.keys(), ac_batch["gt_image"].shape, ac_batch["cond_image"].shape